In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-toxicity-prediction-challenge/sample_submission.csv
/kaggle/input/the-toxicity-prediction-challenge/feamat.csv
/kaggle/input/the-toxicity-prediction-challenge/features_id_name_mappings.csv
/kaggle/input/the-toxicity-prediction-challenge/train.csv
/kaggle/input/the-toxicity-prediction-challenge/test.csv


In [2]:
#Loading and reading required files
train_data = pd.read_csv('/kaggle/input/the-toxicity-prediction-challenge/train.csv')
test_data = pd.read_csv('/kaggle/input/the-toxicity-prediction-challenge/test.csv')
feature_data = pd.read_csv('/kaggle/input/the-toxicity-prediction-challenge/feamat.csv')

In [3]:
train_data.head()

,Id,Expected
0,2971-36-0;1644,2
1,693-54-9;2451,2
2,7173-51-5;1384,2
3,138261-41-3;16,2
4,7681-82-5;1856,2


In [4]:
#Renaming the x column as Id in test dataset
test_data = test_data.rename(columns = {"x": "Id"})
test_data.head()

,Id
0,88-60-8;1682
1,122931-48-0;1656
2,NOCAS_47311;36
3,55589-62-3;1850
4,79902-63-9;30


In [5]:
#Splitting Id into 2 separate columns
train_data[["Chemical_Id", "Assay_Id"]] = train_data.Id.str.split(";", expand = True)
test_data[["Chemical_Id", "Assay_Id"]] = test_data.Id.str.split(";", expand = True)
train_data.head()

,Id,Expected,Chemical_Id,Assay_Id
0,2971-36-0;1644,2,2971-36-0,1644
1,693-54-9;2451,2,693-54-9,2451
2,7173-51-5;1384,2,7173-51-5,1384
3,138261-41-3;16,2,138261-41-3,16
4,7681-82-5;1856,2,7681-82-5,1856


In [6]:
test_data.head()

,Id,Chemical_Id,Assay_Id
0,88-60-8;1682,88-60-8,1682
1,122931-48-0;1656,122931-48-0,1656
2,NOCAS_47311;36,NOCAS_47311,36
3,55589-62-3;1850,55589-62-3,1850
4,79902-63-9;30,79902-63-9,30


In [7]:
#train_mapping = train_data.merge(feature_data, left_on = "Id", how = "left")
train_mapping = pd.merge(train_data, feature_data, left_on = "Chemical_Id", right_on = "V1", how = "left")
test_mapping = pd.merge(test_data, feature_data, left_on = "Chemical_Id", right_on = "V1", how = "left")

In [8]:
#Dropping V1 
train_mapping.drop(["V1", "Id"], axis = 1, inplace = True)
test_mapping.drop(["V1","Id"], axis = 1, inplace = True)

In [9]:
train_mapping.isnull().sum()

Expected       0
Chemical_Id    0
Assay_Id       0
V2             0
V3             0
              ..
V1071          0
V1072          0
V1073          0
V1074          0
V1075          0
Length: 1077, dtype: int64

In [10]:
test_mapping.isnull().values.any()

False

In [11]:
train_mapping.shape, test_mapping.shape

((77413, 1077), (11139, 1076))

In [12]:
train_mapping = train_mapping.drop("Chemical_Id", axis = 1)

In [13]:
train_mapping.head()

,Expected,Assay_Id,V2,V3,V4,V5,V6,V7,V8,V9,...,V1066,V1067,V1068,V1069,V1070,V1071,V1072,V1073,V1074,V1075
0,2,1644,76302,315.982463,4.592,40.46,0.0,0.0,0.000000,0.166667,...,0,0,0,0,0,0,0,0,0,0
1,2,2451,12741,156.151415,3.852,17.07,0.0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,2,1384,23558,361.347528,9.912,0.00,0.0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,2,16,86418,255.052302,2.294,83.66,0.0,0.0,0.117851,0.250000,...,0,0,0,0,0,0,0,0,0,0
4,2,1856,5238,149.894242,1.050,0.00,0.0,0.0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [14]:
train_mapping.replace([np.inf,-np.inf], np.nan,inplace=True)

In [15]:
test_mapping.replace([np.inf,-np.inf],np.nan,inplace=True)

In [16]:
train_mapping.fillna(train_mapping.mean(), inplace=True)

In [17]:
test_mapping.fillna(train_mapping.mean(), inplace=True)

In [18]:
test_mapping.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11139 entries, 0 to 11138
Columns: 1076 entries, Chemical_Id to V1075
dtypes: float64(20), int64(1054), object(2)
memory usage: 91.5+ MB


In [19]:
print(test_mapping.dtypes)

Chemical_Id     object
Assay_Id        object
V2               int64
V3             float64
V4             float64
                ...   
V1071            int64
V1072            int64
V1073            int64
V1074            int64
V1075            int64
Length: 1076, dtype: object


In [20]:
X_train = train_mapping.drop("Expected", axis = 1)
Y_train = train_mapping["Expected"]
X_test  = test_mapping.drop("Chemical_Id", axis=1)

X_train.shape, Y_train.shape, X_test.shape

((77413, 1075), (77413,), (11139, 1075))

In [21]:
X_train["Assay_Id"]=X_train["Assay_Id"].astype(str).astype(float)

In [22]:
#X_train["Assay_Id"]=X_train["Assay_Id"].astype(float).astype(int)

In [23]:
X_test["Assay_Id"]=X_test["Assay_Id"].astype(str).astype(float)

In [24]:
#X_test["Assay_Id"]=X_test["Assay_Id"].astype(float).astype(int)

In [25]:
from sklearn.feature_selection import VarianceThreshold#
var_thres=VarianceThreshold(threshold=0.214)
var_thres.fit(X_train)

VarianceThreshold(threshold=0.214)

In [26]:
sum(var_thres.get_support())

134

In [27]:
len(X_train.columns[(var_thres.get_support())])

134

In [28]:
constant_columns= [column for column in X_train.columns
                  if column not in X_train.columns[var_thres.get_support()]]
print(len(constant_columns))

941


In [29]:
X_train=X_train.drop(constant_columns,axis=1)
X_test=X_test.drop(constant_columns,axis=1)


In [30]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11139 entries, 0 to 11138
Columns: 134 entries, Assay_Id to V905
dtypes: float64(13), int64(121)
memory usage: 11.5 MB


In [31]:
from imblearn.over_sampling import SMOTE

smote=SMOTE(random_state=10)
X_train_smote,Y_train_smote=smote.fit_resample(X_train,Y_train)

In [32]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()  
scaler.fit(X_train_smote)  
X_train_smote = scaler.transform(X_train_smote)  
# apply same transformation to test data
X_test = scaler.transform(X_test) 

In [33]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC


#clf = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
 
clf=XGBClassifier(max_depth=7,n_estimators=665)
#clf=XGBClassifier(Learning_rate=0.1,booster='gbtree',gamma=1.4,max_depth=8,random_state=0,n_estimators=400)


In [34]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from scipy.stats import randint
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score,roc_auc_score
#clf_xgb=XGBClassifier(objective='binary:logistic')
#param_dist={'n_estimators': stats.randint(150,1000),
 #          'learning_rate': stats.uniform(0.01,0.6),
  #         'subsample':stats.uniform(0.3,0.9),
   #         'max_depth':[3,4,5,6,7,8,9],
    #        'colsample_bytree': stats.uniform(0.5,0.9),
     #       'min_child_weight': [1,2,3,4]
      #     }
#clf=RandomizedSearchCV(clf_xgb,param_distributions=param_dist,n_iter=25,verbose=3,n_jobs=-1)


In [35]:
#clf=clf.fit(X_train_smote,Y_train_smote)

In [36]:
#best_score=clf.best_score_
#print("Best Score: {}".format(best_score))

In [37]:
#best_params=clf.best_params_
#print("Best params:")
#for param_name in sorted(best_params.key()):
   # print('%s: %r' %(param_name,best_params[param_name]))

In [38]:
clf.fit(X_train_smote,Y_train_smote)
pred=clf.predict(X_test)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:16:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [39]:
Y_train_smote.head()


0    2
1    2
2    2
3    2
4    2
Name: Expected, dtype: int64

In [40]:
clf.score(X_train_smote,Y_train_smote)

0.992285134220618

In [41]:
sub_test_pred=clf.predict(X_test).astype(int)
sub_test_pred

array([2, 2, 2, ..., 2, 2, 2])

In [42]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
predictions = cross_val_predict(clf, X_train_smote, Y_train_smote, cv = 10)
print(classification_report(Y_train_smote,predictions))

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:21:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:24:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[14:47:22] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:51:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:54:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           1       0.96      0.92      0.94     66495
           2       0.92      0.96      0.94     66495

    accuracy                           0.94    132990
   macro avg       0.94      0.94      0.94    132990
weighted avg       0.94      0.94      0.94    132990



In [43]:
output=pd.DataFrame({'Id': test_mapping["Chemical_Id"].str.cat(test_mapping.Assay_Id.map(str),sep=";"),'predicted': sub_test_pred})
output.to_csv("my.submission.csv",index=False)
print("Submission was succesfull")

Submission was succesfull
